In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

In [2]:
dev = qml.device("default.qubit", wires=["e1", "e2", "e3", "e4", "result"], shots=1)
dev.operations.add("op")

wires = ["e1", "e2", "e3", "e4", "result"]

In [3]:
project=[14,22,26,28]
success=[15,23,27,29]
psi=np.zeros(32,dtype=complex)
for i in range(32):
    if i in project:
        psi[i]=1/2
    #elif i in success:
    #     psi[i]=-1/2/np.sqrt(2)

def densit_matrix(psi):
    L=len(psi)
    M=np.zeros((L,L),dtype=complex)
    for i in range(L):
        for j in range(L):
            M[i,j]=np.conjugate(psi)[i]*psi[j]
    return M

D_s=densit_matrix(psi)

R=2*D_s-np.eye(32)

@qml.qnode(dev)
def circuit(project_execution):
    """This is the circuit we will use to detect which is the lazy worker. Remember 
    that we will only execute one shot.

    Args:
        project_execution (qml.ops): 
            The gate in charge of marking in the last qubit if the project has been finished
            as indicated in the statement.

    Returns:
        (numpy.tensor): Measurement output in the 5 qubits after a shot.
    """
    # Put your code here #
    
    
    def G(p):
        return np.array([[np.sqrt(p), -np.sqrt(1-p)],[np.sqrt(1-p), np.sqrt(p)]])
    
    #U=np.array([[1.0,-1.0],[1.0,1.0]])/np.sqrt(2)
    qml.PauliX(wires="result")
    qml.Hadamard(wires="result")
    qml.PauliX(wires="e1")
    qml.ControlledQubitUnitary(G(1/4), control_wires="e1", wires="e2")
    qml.CNOT(wires=["e2","e1"])
    qml.ControlledQubitUnitary(G(1/3), control_wires="e2", wires="e3")
    qml.CNOT(wires=["e3","e2"])
    qml.ControlledQubitUnitary(G(1/2), control_wires="e3", wires="e4")
    qml.CNOT(wires=["e4","e3"])
    
    qml.PauliX(wires="e1")
    qml.PauliX(wires="e2")
    qml.PauliX(wires="e3")
    qml.PauliX(wires="e4")
    
    project_execution(wires=wires)
    qml.Hadamard(wires="result")
    qml.PauliX(wires="result")
    
    qml.QubitUnitary(R, wires=wires)
    
    # Put your code here #
    return qml.sample(wires=dev.wires)

def process_output(measurement):
    """This function will take the circuit measurement and process it to determine who is the lazy worker.

    Args:
        measurement (numpy.tensor): Measurement output in the 5 qubits after a shot.

    Returns:
        (str): This function must return "e1", "e2" "e3" or "e4" - the lazy worker.
    """
    # Put your code here #
    i=0
    for index in measurement[:4]:
        if index==0:
            goal=i
        i=i+1
    if goal==0:
        return "e1"
    elif goal==1:
        return "e2"
    elif goal==2:
        return "e3"
    elif goal==3:
        return "e4"

In [4]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    return None

def check(solution_output: str, expected_output: str) -> None:
    samples = 5000

    solutions = []
    output = []

    for s in range(samples):
        lazy = np.random.randint(0, 4)
        no_lazy = list(range(4))
        no_lazy.pop(lazy)

        def project_execution(wires):
            class op(qml.operation.Operator):
                num_wires = 5

                def compute_decomposition(self, wires):
                    raise ValueError("You cant descompose this gate")

                def matrix(self):
                    m = np.zeros([32, 32])
                    for i in range(32):
                        b = [int(j) for j in bin(64 + i)[-5:]]
                        if sum(np.array(b)[no_lazy]) == 3:
                            if b[-1] == 0:
                                m[i, i + 1] = 1
                            else:
                                m[i, i - 1] = 1
                        else:
                            m[i, i] = 1
                    return m

            op(wires=wires)
            return None

        out = circuit(project_execution)
        solutions.append(lazy + 1)
        output.append(int(process_output(out)[-1]))

    assert np.allclose(
        output, solutions, rtol=1e-4
    ), "Your circuit does not give the correct output."

    ops = [op.name for op in circuit.tape.operations]
    assert ops.count("op") == 1, "You have used the oracle more than one time."

In [5]:
test_cases = [['No input', 'No output']]

In [6]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input 'No input'...
Correct!
